In [1]:
import cv2,os
data_path = 'D:\data'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict = dict(zip(categories,labels))
                  
print(label_dict)
print(labels)
print(categories)


{'Aircraft  not detected': 0, 'Aircraft detected': 1}
[0, 1]
['Aircraft  not detected', 'Aircraft detected']


In [2]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path) 
    
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)
        
        try:
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            resize = cv2.resize(gray,(img_size,img_size))
            data.append(resize)
            target.append(label_dict[category])
            
            
            
        except Exception as e:
            print('Exception',e)

In [3]:
import numpy as np

data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target = np.array(target)

from keras.utils import np_utils

new_target = np_utils.to_categorical(target)


Using TensorFlow backend.
F:\Users\dell\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Users\dell\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Users\dell\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Users\dell\anaconda3\envs\tensor

In [4]:
np.save('data',data)
np.save('target',new_target)

In [5]:
import numpy as np
data = np.load('data.npy')
target = np.load('target.npy')


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(200,(3,3), input_shape =data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(2))  
model.add(Activation('relu'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='rmsprop', metrics=['accuracy'])


In [7]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [8]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)


Train on 30 samples, validate on 8 samples
Epoch 1/20
30/30 [==============================] - 2s 54ms/step - loss: 0.6919 - accuracy: 0.4333 - val_loss: 1.9611 - val_accuracy: 0.7500
Epoch 2/20
30/30 [==============================] - 1s 41ms/step - loss: 1.4918 - accuracy: 0.8333 - val_loss: 0.6931 - val_accuracy: 0.2500
Epoch 3/20
30/30 [==============================] - 1s 44ms/step - loss: 0.6931 - accuracy: 0.1667 - val_loss: 0.6931 - val_accuracy: 0.2500
Epoch 4/20
30/30 [==============================] - 1s 42ms/step - loss: 0.6931 - accuracy: 0.1667 - val_loss: 0.6931 - val_accuracy: 0.2500
Epoch 5/20
30/30 [==============================] - 1s 42ms/step - loss: 0.6931 - accuracy: 0.1667 - val_loss: 0.6931 - val_accuracy: 0.2500
Epoch 6/20
30/30 [==============================] - 1s 43ms/step - loss: 0.6931 - accuracy: 0.1667 - val_loss: 0.6931 - val_accuracy: 0.2500
Epoch 7/20
30/30 [==============================] - 1s 45ms/step - loss: 0.6931 - accuracy: 0.1667 - val_loss:

In [9]:
from keras.models import load_model
import cv2
import numpy as np

In [10]:
model = load_model('model-001.model')
aircraft_clsf = cv2.CascadeClassifier('D:\jb\craft.xml')
source = cv2.VideoCapture(0)
label_dict={0:'Aircraft  not detected',1:'Aircraft detected'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
while(True):
    
    ret,img = source.read()
   
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    aircrafts = aircraft_clsf.detectMultiScale(gray,1.3,5)
    
    for x,y,w,h in aircrafts:
        
        face_img=gray[y:y+w,x:x+w]
        resize = cv2.resize(face_img,(100,100))
        normalize = resize/255.0
        reshape = np.reshape(normalize,(1,100,100,1))
        result = model.predict(reshape)
        
        label = np.argmax(result,axis=0)[1]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1000)
    if(key==27):
        break
cv2.destroyAllWindows()
source.release()